In [1]:
from extract_features import *
from utils import *
from preprocessing import *
from sklearn.metrics import roc_curve, roc_auc_score
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9529360153903546520
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11285289370
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9055805575942443489
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 5cc5:00:00.0, compute capability: 3.7"
]
['/job:localhost/replica:0/task:0/device:GPU:0']
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5892320181565928352
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11285289370
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13672455238262182027
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 5cc5:00:00.0, compute capability: 3.7"
]
['/job:localhost/replica:0/task:0/device:GPU:0']


In [2]:
embedding_matrix, train_seq, val_seq, y_train, y_val = preprocess(dataset='vqa', 
                                                                  n_classes=2, 
                                                                  verbose=False, 
                                                                  pretrained_embedding=False)
val_data = (val_seq, y_val)

Dataset: vqa 
Features: ['QSN', 'descriptions', 'tags', 'dominant_colors', 'handwritten_text', 'ocr_text']


/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
2019-04-23 11:42:49,119 : INFO : collecting all words and their counts
2019-04-23 11:42:49,120 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-23 11:42:49,138 : INFO : collected 4567 word types from a corpus of 129923 raw words and 3230 sentences
2019-04-23 11:42:49,139 : INFO : Loading a fresh vocabulary
2019-04-23 11:42:49,144 : INFO : effective_min_count=6 retains 845 unique words (18% of original 4567, drops 3722)
2019-04-23 11:42:49,144 : INFO : effective_min_count=6 leaves 124737 word corpus (96% of original 129923, drops 5186)
2019-04-23 11:42:49,147 : INFO : deleting the raw counts dictionary of 4567 items
2019-04-23 11:42:49,148 : INFO : sample=0.001 downsamples 76 most-common words
2019-04-23 11:42:49,149 : INFO : downsamplin

Total 845 word vectors


In [ ]:
L = 1e-3
R = 1e-7
B = 32
E = 500
model, history = skill_predictor(train_seq, embedding_matrix,
                 train_labels=y_train,
                 val_data=(val_seq,y_val),
                 learning_rate=L,
                 lstm_dim=100,
                 batch_size=B,
                 num_epochs=E,
                 optimizer_param=SGD(lr=L, nesterov=True),
                 n_classes=2,
                 regularization=R, verbose=1)

Log file: ./LSTM/0.001_1e-07_32_500.log
Train on 3230 samples, validate on 513 samples
Epoch 1/500
3230/3230 [==============================] - 17s 5ms/step - loss: 0.7401 - acc: 0.5477 - val_loss: 0.6662 - val_acc: 0.6140
Epoch 2/500
3230/3230 [==============================] - 11s 3ms/step - loss: 0.6993 - acc: 0.6108 - val_loss: 0.7150 - val_acc: 0.5721
Epoch 3/500
3230/3230 [==============================] - 11s 3ms/step - loss: 0.6809 - acc: 0.6407 - val_loss: 0.6401 - val_acc: 0.7086
Epoch 4/500
3230/3230 [==============================] - 11s 3ms/step - loss: 0.6668 - acc: 0.6593 - val_loss: 0.6804 - val_acc: 0.6423
Epoch 5/500
3230/3230 [==============================] - 11s 3ms/step - loss: 0.6556 - acc: 0.6738 - val_loss: 0.7453 - val_acc: 0.3402
Epoch 6/500
3230/3230 [==============================] - 11s 3ms/step - loss: 0.6459 - acc: 0.6854 - val_loss: 0.6691 - val_acc: 0.7232
Epoch 7/500
3230/3230 [==============================] - 11s 3ms/step - loss: 0.6377 - acc: 0.697

In [ ]:
preds = model.predict(val_seq, verbose=0)
y_pred_text = [1 if i >= 0.5 else 0 for i in preds[:,0]]
y_pred_color = [1 if i >= 0.5 else 0 for i in preds[:,1]]
print("VizWiz, all features - accuracy\ntext: {}\ncolor:{}".format(accuracy_score(y_val[:,0], y_pred_text),
                                                        accuracy_score(y_val[:,1], y_pred_color)))